In [3]:
#!/usr/bin/env python
#example
import grpc
import rospy
from concurrent import futures
# import the publisher and subscriber node
from ros_bridge import RosBridge
# import the gRPC generated classes
from robo_gym_server_modules.robot_server.grpc_msgs.python import robot_server_pb2, robot_server_pb2_grpc

class Client():

    def __init__(self, server_port):
        self.channel = grpc.insecure_channel('[::]:' + repr(server_port))
        self.robot_server_stub = robot_server_pb2_grpc.RobotServerStub(self.channel)

    def set_state(self, state):
        # Old method, to be gradually replaced in all the stack
        msg = self.robot_server_stub.SetState(robot_server_pb2.State(state = state), timeout = 60)
        return msg.success

    def set_state_msg(self, state_msg):
        msg = self.robot_server_stub.SetState(state_msg, timeout = 60)
        return msg.success

    def get_state(self,):
        # Old method, to be gradually replaced in all the stack
        msg = self.robot_server_stub.GetState(robot_server_pb2.Empty(), timeout = 20)
        if msg.success == 1:
            return msg.state
        else:
            raise Exception('Error while retrieving state')

    def get_state_msg(self,):
        msg = self.robot_server_stub.GetState(robot_server_pb2.Empty(), timeout = 20)
        if msg.success == 1:
            return msg
        else:
            raise Exception('Error while retrieving state')

    def send_action(self, action):
        msg = self.robot_server_stub.SendAction(robot_server_pb2.Action(action = action), timeout = 20 )
        return msg.success

    def send_action_get_state(self, action):
        msg = self.robot_server_stub.SendActionGetState(robot_server_pb2.Action(action = action), timeout = 20 )
        if msg.success == 1:
            return msg
        else:
            raise Exception('Error while retrieving state')


In [6]:
rospy.loginfo('Starting Example Robot Server...')
server_port = 50051
client = Client(server_port)
channel = grpc.insecure_channel('[::]:' + repr(server_port))
robot_server_stub = robot_server_pb2_grpc.RobotServerStub(channel)

In [7]:
state = client.get_state()
print("state: ", state)

state:  [-0.029210390523076057, -0.002061496488749981, 0.0, 0.0]


In [8]:
get_state_msg =client.get_state_msg()
print(get_state_msg)
print("state: ",get_state_msg.state)
print("float_params: ", get_state_msg.float_params)
print("string_params: ", get_state_msg.string_params)
print("state_dict: ", get_state_msg.state_dict)


state: -0.030428683385252953
state: -0.0021111760288476944
state: 0.0
state: 0.0
state_dict {
  key: "ang_vel"
  value: 0.0
}
state_dict {
  key: "lin_vel"
  value: 0.0
}
state_dict {
  key: "pos_x"
  value: -0.030428683385252953
}
state_dict {
  key: "pos_y"
  value: -0.0021111760288476944
}
success: true

state:  [-0.030428683385252953, -0.0021111760288476944, 0.0, 0.0]
float_params:  {}
string_params:  {}
state_dict:  {'pos_x': -0.030428683385252953, 'lin_vel': 0.0, 'ang_vel': 0.0, 'pos_y': -0.0021111760288476944}


In [79]:
action = client.send_action([1, -.1])
action

True

In [86]:
send_action_get_state = client.send_action_get_state([1, -.2])
send_action_get_state

state: 9.422280311584473
state: 7.373431205749512
state: 0.0
state: 0.0
state_dict {
  key: "ang_vel"
  value: 0.0
}
state_dict {
  key: "lin_vel"
  value: 0.0
}
state_dict {
  key: "pos_x"
  value: 9.422280311584473
}
state_dict {
  key: "pos_y"
  value: 7.373431205749512
}
success: true

In [23]:
new_state = {'pos_x': 1.5, 'pos_y': 2}
new_state
new_state_msg = robot_server_pb2.State(state_dict = new_state)
new_state_msg
robot_server_stub.SetState(new_state_msg)

success: true

In [146]:
# State must be dict see rosbridge
new_state = [1, 1, 1, 1]
new_state
new_state_msg = robot_server_pb2.State(state = new_state)
new_state_msg
robot_server_stub.SetState(new_state_msg)

success: true